## Selecting the fingerprints

In this notebook I'm going to select the fingerprints that store the information in a more appropriate manner for a CNN model of 1 dimension. To do that, I'm going to program a script that uses different fingerprints to train a CNN and I will chose the ones giving a model with the best accuracy. The model is build to be fast to speed up the process.

## Imports

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings

### Load data

In [10]:
pd.options.mode.chained_assignment = None

In [11]:
drugs = pd.read_pickle('morgan_and_mac.pkl').drop('FeatInvariants', axis = 1) #We discard the Feature Invariants because they give an error I have to look into yet
drugs.head()

,CID,Molecule,drug_class,drug_class_code,ConnInvariants,Morgan2FP,MACCSKeys,AtomPairFP,TopTorFP,AvalonFP,PubchemFP,CactvsFP
0,24769,<rdkit.Chem.rdchem.Mol object at 0x0000026A61D...,hematologic,7,"[2968968094, 2976033787, 2968968094, 297603378...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
1,134694070,<rdkit.Chem.rdchem.Mol object at 0x0000026A61D...,cardio,3,"[2968968094, 2976033787, 2968968094, 297603378...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
2,5121,<rdkit.Chem.rdchem.Mol object at 0x0000026A61D...,antiinfective,0,"[2968968094, 2976033787, 2968968094, 297603378...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, ..."
3,4660557,<rdkit.Chem.rdchem.Mol object at 0x0000026A61D...,cns,4,"[2968968094, 2976033787, 2968968094, 297603378...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
4,122175,<rdkit.Chem.rdchem.Mol object at 0x0000026A61D...,antineoplastic,2,"[2968968094, 2976033787, 2968968094, 297603378...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, ..."


## Functions

In [12]:
def select_fp(column_list):
    """
    Function that takes a list of columns and makes a train/tesst split for each of them with 'drug_class_code', the column containing the labels, as y.
    Then returns a dictionary with the column name as a key and the tuple with X_train, X_test, y_train, y_test as value.
    Input: list with column names
    Output: dictionary
    """
    splits_dic = {}
    for column in column_list:
        X = drugs[column]
        y = drugs['drug_class_code']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        splits_dic[column] = X_train, X_test, y_train, y_test
    return splits_dic

In [13]:
def build_model(inshape, nclasses):
    """
    Function that builds a CNN model with non optimal conditions.
    Input: tuple with inner shape of arrays, integer with number of classes
    Output: a CNN model
    """
    model = Sequential()
    model.add(Conv1D(100, 9, activation='relu', kernel_initializer='he_uniform', input_shape=inshape))
    model.add(MaxPool1D(2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(nclasses, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [14]:
def reshape_and_build(splits_dic):
    """
    Function that takes the dictionary generated by "select_fp" and reshapes X_train to fit a CNN model.
    Then builds the CNN model using the function 'build_model'. Return both the reshaped arrays and the models.
    Input: dictionary with column names as keys and a tuple of arrays as values
    Output: dictionary with column names as keys and a tuple with both reshaped arrays and CNN models 
    """
    arrays_models_dic = {}
    for column, tup in splits_dic.items():
        print(column)
        X_train = np.array(list(tup[0]))
        X_test = np.array(list(tup[1]))
        n_classes = len(np.unique(tup[2]))
        print('Number of classes: ', n_classes)
        X_train= X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test= X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
        print('Reshaped X_train: ', X_train.shape)
        in_shape = X_train.shape[1:]
        print('In_shape: ', in_shape)
        print('Shape y_train: ', tup[2].shape)
        
        model = build_model(in_shape, n_classes)
        arrays_models_dic[column] = X_train, X_test, tup[2], tup[3], model

    return arrays_models_dic

In [15]:
def fit_model(arrays_models_dic):
    """
    Function that takes the dictionary generated by 'reshape_and_build' and fits the CNN model contained in the values with the X_train and y_train arrays also in the values.
    Then evaluates the model against X_test e y_test and returns the accuracy of each model.
    Input: dictionary with column names as keys and a tuple with reshaped arrays and CNN models
    Output: a dictionary with column names as keys and the accuracy obtained by the CNN model for that column
    """
    accuracies_dic = {}
    es = EarlyStopping(monitor='val_loss', patience=1)
    for column, tup in arrays_models_dic.items():
        print(f"Analysing {column}")
        tup[4].fit(tup[0], tup[2], epochs=10, batch_size=128, verbose=1, validation_split = 0.2, callbacks = [es])
        loss, acc = tup[4].evaluate(tup[1], tup[3], verbose=1)
        accuracies_dic[column] = f'{acc:.3f}'
    return accuracies_dic

## Selection of the best fingerprints

Select the columns containing fingerprints

In [16]:
fp_list = list(drugs.columns[4:])
fp_list

['ConnInvariants',
 'Morgan2FP',
 'MACCSKeys',
 'AtomPairFP',
 'TopTorFP',
 'AvalonFP',
 'PubchemFP',
 'CactvsFP']

Run the functions to obtain the dictionary with the accuracies

In [17]:
# Note for the evaluator: This step may take long,
# you may want to run it partially and see that it works and then skip it and see the result below
# or decrease substantially the number of epochs
splits_dic = select_fp(fp_list)
array_model_dic= reshape_and_build(splits_dic)
accuracies_dic = fit_model(array_model_dic)

ConnInvariants
Number of classes:  12
Reshaped X_train:  (4854, 18, 1)
In_shape:  (18, 1)
Shape y_train:  (4854,)
Morgan2FP
Number of classes:  12
Reshaped X_train:  (4854, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (4854,)
MACCSKeys
Number of classes:  12
Reshaped X_train:  (4854, 167, 1)
In_shape:  (167, 1)
Shape y_train:  (4854,)
AtomPairFP
Number of classes:  12
Reshaped X_train:  (4854, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (4854,)
TopTorFP
Number of classes:  12
Reshaped X_train:  (4854, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (4854,)
AvalonFP
Number of classes:  12
Reshaped X_train:  (4854, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (4854,)
PubchemFP
Number of classes:  12
Reshaped X_train:  (4854, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (4854,)
CactvsFP
Number of classes:  12
Reshaped X_train:  (4854, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (4854,)
Analysing ConnInvariants
Epoch 1/10
31/31 [==============================] - 2s 18ms/step - loss: 26696

31/31 [==============================] - 14s 396ms/step - loss: 3.1876 - accuracy: 0.2660 - val_loss: 1.9407 - val_accuracy: 0.3811
Epoch 2/10
31/31 [==============================] - 11s 367ms/step - loss: 1.8904 - accuracy: 0.3639 - val_loss: 1.6884 - val_accuracy: 0.4212
Epoch 3/10
31/31 [==============================] - 11s 364ms/step - loss: 1.7547 - accuracy: 0.3917 - val_loss: 1.6026 - val_accuracy: 0.4511
Epoch 4/10
31/31 [==============================] - 12s 384ms/step - loss: 1.7578 - accuracy: 0.3953 - val_loss: 1.5638 - val_accuracy: 0.4490
Epoch 5/10
31/31 [==============================] - 12s 373ms/step - loss: 1.6421 - accuracy: 0.4244 - val_loss: 1.4752 - val_accuracy: 0.4768
Epoch 6/10
31/31 [==============================] - 12s 387ms/step - loss: 1.5805 - accuracy: 0.4383 - val_loss: 1.4318 - val_accuracy: 0.5129
Epoch 7/10
31/31 [==============================] - 12s 396ms/step - loss: 1.5838 - accuracy: 0.4316 - val_loss: 1.3865 - val_accuracy: 0.5180
Epoch 8/10

In [18]:
accuracies_dic

{'ConnInvariants': '0.346',
 'Morgan2FP': '0.700',
 'MACCSKeys': '0.496',
 'AtomPairFP': '0.346',
 'TopTorFP': '0.738',
 'AvalonFP': '0.354',
 'PubchemFP': '0.518',
 'CactvsFP': '0.505'}

With 100 epochs and a patiente of 3, the best accuracy is achieved with the Morgan fingerprints. These will be used to tune and train our CNN model.